In [22]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [23]:
year = str(2015)

In [24]:
url_df = pd.read_csv('data/url/'+ year +'_url.csv')

In [35]:
df_user = pd.DataFrame(columns=['user_name', 'game'])
for i in range(854,len(url_df)):
    print(i)
    print(url_df['game'][i])
    # url 가져오기
    url = url_df['url'][i] + '/user-reviews?sort-by=most-helpful&num_items=100'

    #print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # 페이지 수 확인
    last_page_element = soup.find("li", class_="last_page")
    if last_page_element is not None:
        last_page = int(last_page_element.find("a").text)
    else:
        last_page = 1

    users = []
    for page in range(last_page):
        page_url = url + '#&page=' + str(page)

        response = requests.get(page_url, headers=headers)
        html = response.text

        # BeautifulSoup을 사용하여 HTML 문서 파싱
        soup = BeautifulSoup(html, 'html.parser')

        # 사용자 리뷰 엘리먼트를 찾아서 출력
        user = soup.find_all('div', class_='name')
         
        for u in user: users.append(u.get_text(strip=True))
    
    
    for u in users:
        user_url = 'https://www.metacritic.com/user/' + u
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }

        response = requests.get(user_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")

        # 사용자 리뷰 엘리먼트를 찾아서 출력
        title_info = soup.find("ol", class_="reviews user_reviews")

        # title_info에 어떠한 정보라도 있어야 find_all 함수를 사용할 수 있다.
        if title_info is not None:
            titles = title_info.find_all("div", class_="product_title")

            titles_arr = []
            for t in titles:
                titles_arr.append(t.get_text(strip=True))

            if len(titles_arr)>=3:
                user_tmp = pd.DataFrame(columns=['user_name', 'game'])
                user_tmp.loc[0] = [u, titles_arr]
                df_user = pd.concat([df_user, user_tmp])
    
    print(df_user)
    print('******************************************')
    print('******************************************')           

854
Godzilla
         user_name                                               game
0    Gamecriticpro  [Final Fantasy XVI, Hogwarts Legacy, Death Str...
0           lambor  [The Last of Us Part II, Red Dead Redemption 2...
0       zombiechan  [Cyberpunk 2077, Hitman - Episode 2: Sapienza,...
0    SpocksReviews  [Santa's Big Sack, Drumpf: Presidential Dating...
0          SlizzeR  [Godzilla, GRID Autosport, GRID Autosport, GRI...
0          Pureray  [XCOM 2, Armello, Godzilla, Xeodrifter, Grow H...
0      JayLightBar  [God Eater Resurrection, Dragon Ball: Xenovers...
0            Rox22  [Dark Souls III: The Fire Fades Edition, Dark ...
0     godzillabots  [Sonic Frontiers, Jurassic World Evolution 2, ...
0      flametail23  [Death Stranding, Dauntless, Godzilla, The Wit...
0         Euknowia  [State of Decay 2, Extinction, Sea of Thieves,...
0     jordancotton  [Call of Duty: Modern Warfare, Borderlands 3, ...
0     FriedConsole  [Scintillatron 4096, Scintillatron 4096, Necro...
0      

In [36]:
df_user_2 = df_user.drop_duplicates(subset='user_name')

In [37]:
# 새로운 데이터프레임 생성
new_df = pd.DataFrame(columns=['user_name', 'game'])

# 기존 데이터프레임을 행 별로 반복하며 변경
for index, row in df_user_2.iterrows():
    user_name = row['user_name']
    games = row['game']
    
    for game in games:
        new_row = {
            'user_name': user_name,
            'game': game
        }
        new_row_df = pd.DataFrame([new_row])
        new_df = pd.concat([new_df, new_row_df])

In [38]:
# 중복 행 제거
new_df = new_df.drop_duplicates(['user_name', 'game'])

# 인덱스 재정렬
new_df = new_df.reset_index(drop=True)

In [39]:
from datetime import datetime

# 현재 시간을 파일 이름으로 사용
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# CSV 파일로 저장
filename = year + f'_data_{current_time}.csv'
new_df.to_csv('data/'+ filename + '_user_log_sample.csv', index=False)